# Import các thư viện cần thiết

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Lấy dữ liệu

In [35]:
df = pd.read_csv("/content/vietnam_housing_dataset.csv")
df.head()

,Address,Area,Frontage,Access Road,House direction,Balcony direction,Floors,Bedrooms,Bathrooms,Legal status,Furniture state,Price
0,"Dự án The Empire - Vinhomes Ocean Park 2, Xã L...",84.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,Have certificate,NaN,8.60
1,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...",60.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,7.50
2,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...",90.0,6.0,13.0,Đông - Bắc,Đông - Bắc,5.0,NaN,NaN,Sale contract,NaN,8.90
3,"Đường Nguyễn Văn Khối, Phường 11, Gò Vấp, Hồ C...",54.0,NaN,3.5,Tây - Nam,Tây - Nam,2.0,2.0,3.0,Have certificate,Full,5.35
4,"Đường Quang Trung, Phường 8, Gò Vấp, Hồ Chí Minh",92.0,NaN,NaN,Đông - Nam,Đông - Nam,2.0,4.0,4.0,Have certificate,Full,6.90


In [36]:
# Loại bỏ cột không dùng hoặc chứa quá nhiều giá trị khác biệt (Address)
df = df.drop(columns=["Address"])

# Loại bỏ hàng thiếu dữ liệu
df = df.dropna()
# Mã hoá biến phân loại
categorical_cols = ["House direction", "Balcony direction", "Legal status", "Furniture state"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)



In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2645 entries, 16 to 30220
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Area                          2645 non-null   float64
 1   Frontage                      2645 non-null   float64
 2   Access Road                   2645 non-null   float64
 3   Floors                        2645 non-null   float64
 4   Bedrooms                      2645 non-null   float64
 5   Bathrooms                     2645 non-null   float64
 6   Price                         2645 non-null   float64
 7   House direction_Nam           2645 non-null   bool   
 8   House direction_Tây           2645 non-null   bool   
 9   House direction_Tây - Bắc     2645 non-null   bool   
 10  House direction_Tây - Nam     2645 non-null   bool   
 11  House direction_Đông          2645 non-null   bool   
 12  House direction_Đông - Bắc    2645 non-null   bool   
 13  House 

In [40]:
# Chuẩn bị biến đặc trưng và nhãn
X = df.drop(columns=["Price"]).values
y = df["Price"].values.reshape(-1, 1)

In [46]:
# Chuẩn hoá dữ liệu (rất quan trọng cho Gradient Descent)

# Convert boolean columns to integer (0 or 1)
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)

X = df.drop(columns=["Price"]).values
y = df["Price"].values.reshape(-1, 1)

X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean()) / y.std()

In [49]:
class LinearRegressionGD:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.theta = None
        self.losses = []

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y).reshape(-1, 1)
        m, n = X.shape
        X_b = np.c_[np.ones((m, 1)), X]
        self.theta = np.zeros((n + 1, 1))

        for epoch in range(self.epochs):
            y_pred = X_b.dot(self.theta)
            error = y_pred - y
            gradients = (1/m) * X_b.T.dot(error)
            self.theta -= self.learning_rate * gradients

            loss = (1/(2*m)) * np.sum(error ** 2)
            self.losses.append(loss)

        return self

    def predict(self, X):
        X = np.array(X)
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b.dot(self.theta)

    def score(self, X, y):
        y_pred = self.predict(X).flatten()
        y = np.array(y).flatten()
        SS_res = np.sum((y - y_pred) ** 2)
        SS_tot = np.sum((y - np.mean(y)) ** 2)
        return 1 - SS_res / SS_tot


In [55]:
model = LinearRegressionGD(learning_rate=0.01, epochs=10000)
model.fit(X, y)

print("R² =", model.score(X, y))
print("Số đặc trưng:", X.shape[1])
print("Trọng số đầu tiên:", model.theta[:5])


R² = 0.2980884681993985
Số đặc trưng: 22
Trọng số đầu tiên: [[-4.69886679e-17]
 [ 1.78624142e-01]
 [-1.57030235e-02]
 [ 1.68802136e-01]
 [ 1.45984881e-01]]
